In [ ]:
cd ..

In [ ]:
import os

In [ ]:
import numpy as np

In [ ]:
import echofilter.raw

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

In [ ]:
turbulence_color = "c"
bottom_color = "#00dd00"

In [ ]:
ROOT_DATA_DIR = "/media/scott/scratch/Datasets/dsforce/surveyExports"
dataset = "MinasPassage"
# has removed window
sample = "december2017/december2017_D20171214-T202211_D20171215-T015215"
# has passive recording
sample = "september2018/september2018_D20181116-T205220_D20181117-T022218"

In [ ]:
sv_path = os.path.join(ROOT_DATA_DIR, dataset, sample + "_Sv_raw.csv")
evl_path = os.path.join(ROOT_DATA_DIR, dataset, sample + "_air.evl")

In [ ]:
timestamps, depths, signals = echofilter.raw.loader.transect_loader(sv_path)

In [ ]:
print(timestamps.shape)
print(depths.shape)
print(signals.shape)

In [ ]:
timestamps

In [ ]:
depths

In [ ]:
min(depths), max(depths)

In [ ]:
signals

In [ ]:
t_top, d_top = echofilter.raw.loader.evl_loader(evl_path)

In [ ]:
print(t_top.shape)
print(d_top.shape)

In [ ]:
t_top

In [ ]:
d_top

In [ ]:
min(d_top), max(d_top)

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(timestamps, depths, signals.T)
# plt.plot(t_bot, d_top, 'c')
plt.gca().invert_yaxis()
plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
plt.plot(t_top, d_top, "b")
plt.show()

## Splitting stationary recordings into continguous periods with gaps between them

In [ ]:
dt = np.diff(timestamps)
min(dt), max(dt)

In [ ]:
np.median(dt)

In [ ]:
sum(dt > np.median(dt) * 50)

In [ ]:
break_indices = np.where(dt > np.median(dt) * 50)[0]

if len(break_indices) > 0:
    break_indices += 1

break_indices

In [ ]:
timestamps[296:302] - timestamps[0]

In [ ]:
timestamps[596:600] - timestamps[0]

In [ ]:
for chunk_start, chunk_end in zip(
    np.concatenate(([0], break_indices)),
    np.concatenate((break_indices, [len(timestamps)])),
):
    plt.figure(figsize=(12, 12))
    plt.pcolormesh(
        timestamps[chunk_start:chunk_end], depths, signals[chunk_start:chunk_end, :].T
    )
    li = np.logical_and(
        timestamps[chunk_start] <= t_top, t_top <= timestamps[chunk_end - 1]
    )
    plt.plot(t_top[li], d_top[li], turbulence_color)
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
depths[0] - depths[-1]

## Try using existing mask decomposer

In [ ]:
transect = echofilter.raw.manipulate.load_decomposed_transect_mask(
    os.path.join(ROOT_DATA_DIR, dataset, sample),
    dataset,
)

In [ ]:
transect

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(transect["timestamps"], transect["depths"], transect["Sv"].T)
plt.plot(transect["timestamps"], transect["top"], turbulence_color)
plt.plot(transect["timestamps"], transect["bottom"], bottom_color)
plt.gca().invert_yaxis()
plt.xlabel("Timestamp (s)")
plt.ylabel("Depth (m)")
plt.show()

In [ ]:
for i_chunk, (ck_start, ck_end) in enumerate(
    zip(
        np.concatenate(([0], break_indices)),
        np.concatenate((break_indices, [len(timestamps)])),
    )
):
    for signal_name in ("Sv", "mask"):
        plt.figure(figsize=(12, 12))
        plt.pcolormesh(
            transect["timestamps"][ck_start:ck_end],
            transect["depths"],
            transect[signal_name][ck_start:ck_end, :].T,
        )
        plt.plot(
            transect["timestamps"][ck_start:ck_end],
            transect["top"][ck_start:ck_end],
            turbulence_color,
        )
        plt.plot(
            transect["timestamps"][ck_start:ck_end],
            transect["bottom"][ck_start:ck_end],
            bottom_color,
        )

        indices = np.nonzero(transect["is_passive"][ck_start:ck_end])[0]
        if len(indices) > 0:
            r_starts = [indices[0]]
            r_ends = [indices[-1]]
            breaks = np.nonzero(indices[1:] - indices[:-1] > 1)[0]
            for break_idx in breaks:
                r_ends.append(indices[break_idx + 1])
                r_starts.append(indices[break_idx + 2])
            for r_start, r_end in zip(r_starts, r_ends):
                plt.fill_between(
                    transect["timestamps"][ck_start:ck_end][[r_start, r_end]],
                    transect["depths"][[0, 0]],
                    transect["depths"][[-1, -1]],
                    facecolor="none",
                    hatch="//",
                    edgecolor=[0.4, 0.4, 0.4],
                    linewidth=0.0,
                )

        indices = np.nonzero(transect["is_removed"][ck_start:ck_end])[0]
        if len(indices) > 0:
            r_starts = [indices[0]]
            r_ends = [indices[-1]]
            breaks = np.nonzero(indices[1:] - indices[:-1] > 1)[0]
            for break_idx in breaks:
                r_ends.append(indices[break_idx + 1])
                r_starts.append(indices[break_idx + 2])
            for r_start, r_end in zip(r_starts, r_ends):
                plt.fill_between(
                    transect["timestamps"][ck_start:ck_end][[r_start, r_end]],
                    transect["depths"][[0, 0]],
                    transect["depths"][[-1, -1]],
                    facecolor="none",
                    hatch="\\\\",
                    edgecolor=[0, 0, 1],
                    linewidth=0.0,
                )

        plt.gca().invert_yaxis()
        plt.xlabel("Timestamp (s)")
        plt.ylabel("Depth (m)")
        plt.title("Chunk {}".format(i_chunk))
        plt.show()

In [ ]:
plt.figure(figsize=(12, 12))
plt.pcolormesh(
    transect["timestamps"][ck_start:ck_end],
    transect["depths"][:200],
    transect["Sv"][ck_start:ck_end, :200].T,
)
# plt.plot(transect['timestamps'][ck_start:ck_end], transect['top'][ck_start:ck_end], turbulence_color)
plt.plot(
    transect["timestamps"][ck_start:ck_end],
    transect["bottom"][ck_start:ck_end],
    bottom_color,
)

plt.gca().invert_yaxis()
plt.xlabel("Timestamp (s)")
plt.ylabel("Depth (m)")
plt.show()

In [ ]:
ck_start = break_indices[6]
ck_end = break_indices[7]

plt.figure(figsize=(12, 12))
plt.pcolormesh(
    transect["timestamps"][ck_start:ck_end],
    transect["depths"][:200],
    transect["Sv"][ck_start:ck_end, :200].T,
)
# plt.plot(transect['timestamps'][ck_start:ck_end], transect['top'][ck_start:ck_end], turbulence_color)
# plt.plot(transect['timestamps'][ck_start:ck_end], transect['bottom'][ck_start:ck_end], bottom_color)

plt.gca().invert_yaxis()
plt.xlabel("Timestamp (s)")
plt.ylabel("Depth (m)")
plt.show()